# 神经网络凝聚现象
凝聚现象是指在神经网络的训练过程中，同一层神经元逐渐变得相似。例如，多个神经元最终可能对同一个输入产生非常相似的输出。尽管神经元一开始是随机初始化的，但通过训练，它们有趋于一致的倾向。这种现象揭示了神经网络内部的某种协调机制，有助于我们更好地理解神经网络的运作方式。我们将在这份代码中以全连接网络拟合一个一维函数的例子来说明这个现象。

# 相关文献
[1] Tao Luo#, Zhi-Qin John Xu #, Zheng Ma, Yaoyu Zhang*, Phase diagram for two-layer ReLU neural networks at infinite-width limit, arxiv 2007.07497 (2020), Journal of Machine Learning Research (2021) [pdf](https://ins.sjtu.edu.cn/people/xuzhiqin/pub/phasediagram2020.pdf), and in [arxiv](https://arxiv.org/abs/2007.07497). 

[2] Hanxu Zhou, Qixuan Zhou, Tao Luo, Yaoyu Zhang*, Zhi-Qin John Xu*, Towards Understanding the Condensation of Neural Networks at Initial Training. arxiv 2105.11686 (2021) [pdf](https://ins.sjtu.edu.cn/people/xuzhiqin/pub/initial2105.11686.pdf), and in [arxiv](https://arxiv.org/abs/2105.11686), see slides and [video talk in Chinese](https://www.bilibili.com/video/BV1tb4y1d7CZ/?spm_id_from%253D333.999.0.0), NeurIPS2022. 

For more details, see [xuzhiqin condense](https://ins.sjtu.edu.cn/people/xuzhiqin/pubcondense.html)

In [37]:
# 在这里我们导入一些常用的库
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List
import argparse
import matplotlib.pyplot as plt
import datetime
import re
import copy


# Condense 
![Condense](./pic/condense.png)
这是一个理想的凝聚的示意图。在初始化时，各个神经元的输入权重差异很大，用不同的颜色表示。但是在经过一段时间训练后，中间隐藏神经元分成了两类，前两个神经元是一类，后三个神经元是另一类。在每一类中，不同神经元的输入权重是完全一样的~(颜色相同)，因此，它们的输出也是一样的。

#   一些基础设定的参数
注意要在ini_path中加入自己保存实验结果的地址。

In [38]:
parser = argparse.ArgumentParser(description='PyTorch 1D dataset Training')



parser.add_argument('--lr', default=0.00001, type=float, help='learning rate')  # 学习率
parser.add_argument('--optimizer', default='adam', help='optimizer: sgd | adam')  # 优化器选择：sgd 或 adam
parser.add_argument('--epochs', default=2000, type=int, metavar='N', help='number of total epochs to run')  # 总训练轮数
parser.add_argument('--test_size', default=10000, type=int, help='the test size for model (default: 10000)')  # 测试集大小
parser.add_argument('--save', default='trained_nets', help='path to save trained nets')  # 保存训练模型的路径
parser.add_argument('--save_epoch', default=10, type=int, help='save every save_epochs')  # 每多少轮保存一次模型
parser.add_argument('--rand_seed', default=0, type=int, help='seed for random num generator')  # 随机数生成器的种子
parser.add_argument('--gamma', type=float, default=2, help='parameter initialization distribution variance power(We first assume that each layer is the same width.)')  # 参数初始化分布方差幂（假设每层宽度相同）
parser.add_argument('--boundary', nargs='+', type=str, default=['-1', '1'], help='the boundary of 1D data')  # 一维数据的边界
parser.add_argument('--training_size', default=80, type=int, help='the training size for model (default: 1000)')  # 训练集大小
parser.add_argument('--act_func_name', default='Tanh', help='activation function')  # 激活函数名称
parser.add_argument('--hidden_layers_width', nargs='+', type=int, default=[50])  # 隐藏层宽度
parser.add_argument('--input_dim', default=5, type=int, help='the input dimension for model (default: 1)')  # 模型输入维度
parser.add_argument('--output_dim', default=1, type=int, help='the output dimension for model (default: 1)')  # 模型输出维度
parser.add_argument('--device', default='cuda', type=str, help='device used to train (cpu or cuda)')  # 训练使用的设备（CPU 或 CUDA）
parser.add_argument('--plot_epoch', default=100, type=int, help='step size of plotting interval (default: 1000)')  # 绘图间隔的步长
parser.add_argument('--ini_path', default='/home/zhouzhangchen/condensation', type=str, help='the path to save experiment results')  # 保存实验结果的路径

args, unknown = parser.parse_known_args()


#   实验结果的目录

In [39]:
def mkdirs(fn):  # Create directorys
    if not os.path.isdir(fn):
        os.makedirs(fn)
    return fn


def create_save_dir(path_ini): 
    subFolderName = re.sub(r'[^0-9]', '', str(datetime.datetime.now()))
    path = os.path.join(path_ini, subFolderName)
    mkdirs(path)
    # mkdirs(os.path.join(path, 'output'))
    return path


args.path = create_save_dir(args.ini_path)


# 生成用于训练和测试的数据
我们使用的目标函数为：
\begin{equation}
f(x)=\sum\limits_{k=1}^53.5\sin(5x_k+1)
\end{equation}
其中数据$\boldsymbol{x} = (x_1,x_2,x_3,x_4,x_5)$， 从$[-4,2]$中均匀采样得到。


In [40]:
def get_y(x):  # Function to fit
    y = 3.5 * torch.sum(torch.sin(5*x+1),dim=1)
    y = torch.unsqueeze(y, 1)
    return y

for i in range(2):
    if isinstance(args.boundary[i], str):
        args.boundary[i] = eval(args.boundary[i])

# args.test_input = torch.reshape(torch.linspace(args.boundary[0], args.boundary[1], steps=args.test_size), [args.test_size, 1])
args.test_input = torch.rand((args.test_size, args.input_dim)) * 6 - 4

# args.training_input = torch.reshape(torch.linspace(args.boundary[0], args.boundary[1], steps=args.training_size), [args.training_size, 1])
args.training_input = torch.rand((args.training_size, args.input_dim)) * 6 - 4
args.test_target = get_y(args.test_input)
args.training_target = get_y(args.training_input)



# 定义一些不太常用的激活函数
我们在这里定义$f(x)= x * \mathrm{tanh} (x)$，具体用处在后续给出。

In [41]:
class xtanh(nn.Module):
    def __init__(self):
        super(xtanh,self).__init__()
    
    def forward(self,x):
        return x * nn.Tanh()(x) 


# 神经网络模型及其初始化

首先，给定参数 $\theta\in \mathbb{R}^M$，FNN函数 $f_{\theta}(\cdot)$ 是递归定义的。我们从输入层开始，定义 $f^{[0]}_{\theta}(x)=x$，其中 $x\in\mathbb{R}^d$ 是输入数据。
接下来，对于每一层 $l\in[L-1]$，我们递归地定义 $f^{[l]}_{\theta}$ 如下：
$f^{[l]}{\theta}(x)=\sigma (W^{[l]} f^{[l-1]}{\theta}(x)+b^{[l]})$
这里，$\sigma$ 是一个非线性激活函数，$W^{[l]}$ 是权重矩阵，$b^{[l]}$ 是偏置向量。
最后，我们定义整个网络的输出为：
\begin{equation*}
f_{\theta}(x)=f(x,\theta)=f^{[L]}{\theta}(x)=W^{[L]} f^{[L-1]}{\theta}(x)+b^{[L]}.
\end{equation*}
注意，输出层通常不使用激活函数。
现在，让我们来看看参数的初始化方法。我们使用带有参数 $\gamma$ 的高斯分布来初始化参数，具体如下：
\begin{equation*}
\theta_{l} \sim N(0, \frac{1}{m_{l}^{\gamma}}),
\end{equation*}
这里，$\theta_{l}$ 表示第 $l$ 层的参数，它是一个有序对 $\theta_l=\Big(W^{[l]},b^{[l]}\Big),\quad l\in[L]$。$m_{l}$ 是第 $l$ 层的宽度（即神经元数量）。

In [42]:
class Linear(nn.Module):
    def __init__(self, gamma, hidden_layers_width=[100],  input_size=20, num_classes: int = 1000, act_layer: nn.Module = nn.ReLU()):
        super(Linear, self).__init__()
        self.num_classes = num_classes
        self.input_size = input_size
        self.hidden_layers_width = hidden_layers_width
        self.gamma = gamma
        layers: List[nn.Module] = []
        self.layers_width = [self.input_size]+self.hidden_layers_width
        for i in range(len(self.layers_width)-1):
            layers += [nn.Linear(self.layers_width[i],
                                    self.layers_width[i+1]), act_layer]
        layers += [nn.Linear(self.layers_width[-1], num_classes, bias=False)]
        self.features = nn.Sequential(*layers)
        self._initialize_weights()


    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.features(x)
        return x

    def _initialize_weights(self) -> None:

        for obj in self.modules():
            if isinstance(obj, (nn.Linear, nn.Conv2d)):
                nn.init.normal_(obj.weight.data, 0, 1 /
                                self.hidden_layers_width[0]**(self.gamma))
                if obj.bias is not None:
                    nn.init.normal_(obj.bias.data, 0, 1 /
                                    self.hidden_layers_width[0]**(self.gamma))


def get_act_func(act_func):
    if act_func == 'Tanh':
        return nn.Tanh()
    elif act_func == 'ReLU':
        return nn.ReLU()
    elif act_func == 'Sigmoid':
        return nn.Sigmoid()
    elif act_func == 'xTanh':
        return xtanh()
    else:
        raise NameError('No such act func!')


act_func = get_act_func(args.act_func_name)

model = Linear(args.gamma, args.hidden_layers_width, args.input_dim,
               args.output_dim, act_func).to(args.device)

para_init = copy.deepcopy(model.state_dict())


# 定义一步训练的函数

在训练过程中，我们需要定义一个损失函数来衡量模型的表现。这里我们使用经验风险（empirical risk）作为我们的损失函数：
\begin{equation*}
R_S(\theta)=\frac{1}{n}\sum_{i=1}^n\ell(f(x_i,\theta),y(x_i)),
\end{equation*}
这个公式中，$\ell(\cdot,\cdot)$ 是一个可微的损失函数，用来衡量预测值 $f(x_i,\theta)$ 和真实值 $y(x_i)$ 之间的差距。我们用 $\nabla\ell(y,y^*)$ 表示 $\ell$ 对其第一个参数的导数。
现在，我们的目标是最小化这个经验风险。我们可以使用梯度下降法来更新模型参数。对于一步梯度下降，参数更新公式如下：
\begin{equation*}
\theta_{t+1}=\theta_t-\eta\nabla R_S(\theta).
\end{equation*}
这里，$\theta_t$ 是当前的参数值，$\theta_{t+1}$ 是更新后的参数值，$\eta$ 是学习率（一个小的正数），$\nabla R_S(\theta)$ 是经验风险关于参数 $\theta$ 的梯度。

In [43]:
def train_one_step(model, optimizer, loss_fn,  args):

    model.train()
    device = args.device
    data, target = args.training_input.to(
        device), args.training_target.to(device).to(torch.float)

    optimizer.zero_grad()
    outputs = model(data)
    loss = loss_fn(outputs, target)
    loss.backward()
    optimizer.step()

    return loss.item()


# 定义一步测试的函数

In [44]:
def test(model, loss_fn, args):
    model.eval()
    device = args.device
    with torch.no_grad():
        data, target = args.test_input.to(
            device), args.test_target.to(device).to(torch.float)
        outputs = model(data)
        loss = loss_fn(outputs, target)

    return loss.item(), outputs


# 定义可视化每一步损失的函数

In [45]:
def plot_loss(path, loss_train, x_log=False):

    plt.figure()
    ax = plt.gca()
    y2 = np.asarray(loss_train)
    plt.plot(y2, 'k-', label='Train')
    plt.xlabel('epoch', fontsize=18)
    ax.tick_params(labelsize=18)
    plt.yscale('log')
    if x_log == False:
        fntmp = os.path.join(path, 'loss.jpg')

    else:
        plt.xscale('log')
        fntmp = os.path.join(path, 'loss_log.jpg')
    plt.tight_layout()
    plt.savefig(fntmp,dpi=300)


    plt.close()


# 定义可视化神经网络输出的函数


In [46]:
def plot_model_output(path, args, output, epoch):

    plt.figure()
    ax = plt.gca()

    plt.plot(args.training_input.detach().cpu().numpy(),
             args.training_target.detach().cpu().numpy(), 'b*', label='True')
    plt.plot(args.test_input.detach().cpu().numpy(),
             output.detach().cpu().numpy(), 'r-', label='Test')

    ax.tick_params(labelsize=18)
    plt.legend(fontsize=18)
    fn = mkdirs(os.path.join('%s'%path,'output'))
    fntmp = os.path.join(fn, str(epoch)+'.jpg')

    plt.savefig(fntmp, dpi=300)


    plt.close()


# 定义函数可视化训练过程中高维神经元（$d>1$）的特征

对于高维数据,无法使用极坐标系。在这种情况下,我们可以利用余弦相似度来衡量两个向量之间夹角的大小。

**余弦相似度**: 两个向量 $\boldsymbol{u}$ 和 $\boldsymbol{v}$ 的余弦相似度定义为
\begin{equation}
D(\boldsymbol{u},\boldsymbol{v}) = \frac{\boldsymbol{u}^T\boldsymbol{v}}{(\boldsymbol{u}^T\boldsymbol{u})^{1/2}(\boldsymbol{v}^{T}\boldsymbol{v})^{1/2}}.
\end{equation}

In [47]:
def get_parameter(checkpoint):
    wei1 = checkpoint['features.0.weight']
    bias = checkpoint['features.0.bias']
    wei2 = checkpoint['features.2.weight']

    return wei1, bias, wei2

def normalize_vectorgroup(checkpoint):
    wei1, bias, wei2 = get_parameter(checkpoint)
    bias = torch.unsqueeze(bias,dim=1)
    vector_group = torch.cat((wei1,bias),dim=1)
    vector_group = vector_group.detach().cpu().numpy()
    norms = np.linalg.norm(vector_group,axis=1)
    mask = norms > 0
    vector_masked = vector_group[mask]
    norms = norms[mask]
    norms = norms[:, np.newaxis]
    vector_normalized = vector_masked / norms
    return vector_normalized,vector_masked.shape[0]


def seperate_vectors_by_eigenvector(vector_group):
    mask = np.linalg.norm(vector_group,axis=1) > 0
    vector_group = vector_group[mask]
    similarity_matrix = np.dot(vector_group,vector_group.transpose())
    w,v = np.linalg.eig(similarity_matrix)
    index = np.argmax(w)
    tmpeig = v[:,index]
    order_mask = np.argsort(tmpeig)
    
    similarity_matrix = similarity_matrix[order_mask,:]
    similarity_matrix = similarity_matrix[:,order_mask]
    return similarity_matrix,order_mask

def plot_weight_heatmap_eigen(weight, path, args, nota=''):

    weight_normalized,masked_shape = normalize_vectorgroup(weight)
    similarity_matrix,order = seperate_vectors_by_eigenvector(weight_normalized)
    fn = mkdirs(os.path.join('%s'%path,'cosine_similarity'))
    plt.figure()
    plt.pcolormesh(similarity_matrix,vmin=-1,vmax=1,cmap='YlGnBu')
    plt.colorbar()
    plt.xlabel('index',fontsize=18)
    plt.xticks(fontsize=18)
    plt.ylabel('index',fontsize=18)
    plt.yticks(fontsize=18)
    plt.tight_layout()
    plt.savefig(os.path.join(fn,'%s'%nota))
    plt.close()
    return order

# 进行训练的函数
有了前面的关于神经网络的基础以及一些关于训练过程函数的定义之后，我们可以对神经网络进行训练，并可视化训练过程中我们关心的一些量。

In [ ]:

args.gamma = 4
args.lr = 0.001
args.epochs = 100
args.save_epoch = 10
args.plot_epoch = 10
args.optimizer = 'adam'
args.act_func_name = 'Tanh'
args.savepath = os.path.join(args.path, 't=%s'%args.gamma)
os.makedirs(args.savepath, exist_ok=True)
act_func = get_act_func(args.act_func_name)

model = Linear(args.gamma, args.hidden_layers_width, args.input_dim,
               args.output_dim, act_func).to(args.device)

para_init = copy.deepcopy(model.state_dict())
if args.optimizer=='sgd':
  optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
else:
  optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
loss_fn = nn.MSELoss(reduction='mean')
t0 = time.time()
loss_training_lst=[]
loss_test_lst = []
for epoch in range(args.epochs+1):

      model.train()
      loss = train_one_step(
        model, optimizer, loss_fn, args)
      loss_test, output = test(
          model, loss_fn, args)
      loss_training_lst.append(loss)
      loss_test_lst.append(loss_test)
      if epoch % args.plot_epoch == 0:
            print("[%d] loss: %.6f valloss: %.6f time: %.2f s" %
                  (epoch + 1, loss, loss_test, (time.time()-t0)))
  
      if (epoch+1) % (args.plot_epoch) == 0:
          plot_loss(path=args.savepath,
                    loss_train=loss_training_lst, x_log=True)
          plot_loss(path=args.savepath,
                    loss_train=loss_training_lst, x_log=False)

          
          para_now = copy.deepcopy(model.state_dict())
          plot_weight_heatmap_eigen(para_now, args.savepath, args, nota='%s'%epoch)
          
